In [1]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import pymongo
from splinter import Browser
import pandas as pd
import time
import re

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

# NASA Mars News

In [9]:
#Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text
#to variables that you can reference later.
#URL of page to be scraped
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

#Pulling in data
html = browser.html

# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(html, 'html.parser')

In [10]:
# find the first news title
news_title = soup.body.find("div", class_="content_title").text

# find the paragraph associated with the first title
news_paragraph = soup.body.find("div", class_="article_teaser_body").text

print(f"{news_title}")
print(f"{news_paragraph}")

Mars Now
The team also fueled the rover's sky crane to get ready for this summer's history-making launch.


# JPL Mars Space Images - Featured Image 

In [11]:
#URL of page to be scraped
mars_image_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"

browser.visit(mars_image_url)

browser.click_link_by_id('full_image')

# click on the more info button to get to the large image

browser.click_link_by_partial_text('more info')

/Users/rubenbenitez/opt/anaconda3/lib/python3.7/site-packages/splinter/driver/webdriver/__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [12]:

# create the soup item

image_html = browser.html
mars_image_soup = BeautifulSoup(image_html, 'html.parser')

# the large image is within the figue element with class = lede
image = mars_image_soup.body.find("figure", class_="lede")
    
# the url is within the a element, so search for a element and then extract the url
link = image.find('a')
href = link['href']

# define the beginning of the url as the returned href doesn't included it
base_url='https://www.jpl.nasa.gov'

# create the full url
featured_image_url = base_url + href

featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA19083_hires.jpg'

# Mars Weather 

In [39]:
#Mars Weather
#URL of page to be scraped
url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(url)
time.sleep(5)
#Pulling in data
html = browser.html

# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(html, 'html.parser')

#Tweet text
mars_weather = soup.find("div", attrs={"class":"tweet", "data":"Mars Weather"})                         
                         #class_="css-901oao").text
    #css-901oao css-16my406 r-1qd0xha r-ad9z0x r-bcqeeo r-qvutc0
try:
    m_weather = mars_weather.find("p", "tweet.text").get_text()
    m_weather
except AttributeError:
    pattern = re.compile(r'sol')
    m_weather = soup.find("span",text=pattern).text
    m_weather
m_weather

'InSight sol 484 (2020-04-06) low -94.3ºC (-137.8ºF) high -7.0ºC (19.3ºF)\nwinds from the SW at 5.4 m/s (12.1 mph) gusting to 17.3 m/s (38.7 mph)\npressure at 6.50 hPa'

# Mars Facts

In [40]:
# Mars facts to be scraped
facts_url = 'https://space-facts.com/mars/'
tables = pd.read_html(facts_url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
           

In [41]:
mars_facts_df = tables[2]
mars_facts_df.columns = ["Description", "Value"]
mars_facts_df

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [42]:
mars_html_table = mars_facts_df.to_html()
mars_html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Description</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium 

In [43]:

mars_html_table.replace('\n', '')
print(mars_html_table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Description</th>
      <th>Value</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
    

# Mars Hemispheres

In [ ]:
# Mars hemisphere name and image to be scraped
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemispheres_url)
hemispheres_html = browser.html
hemispheres_soup = BeautifulSoup(hemispheres_html, 'html.parser')

In [ ]:
#Mars Hemispheres
#URL of page to be scraped - Cerberus
url = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/cerberus_enhanced'
browser.visit(url)

#Pulling in data
html = browser.html

# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(html, 'html.parser')

#Image url
cerberus_url = soup.find('div', class_='downloads')
link = cerberus_url.find('a')
cerberus_href = link['href']

print(cerberus_href) 

In [ ]:
#Mars Hemispheres
#URL of page to be scraped - Schiaparelli
url = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/schiaparelli_enhanced'
browser.visit(url)

#Pulling in data
html = browser.html

# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(html, 'html.parser')

#Image url
schia_url = soup.find('div', class_='downloads')
link = schia_url.find('a')
schia_href = link['href']

print(schia_href)

In [ ]:
#Mars Hemispheres
#URL of page to be scraped - Syrtis
url = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/syrtis_major_enhanced'
browser.visit(url)

#Pulling in data
html = browser.html

# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(html, 'html.parser')

#Image url
syrtis_url = soup.find('div', class_='downloads')
link = syrtis_url.find('a')
syrtis_href = link['href']

print(syrtis_href)

In [ ]:
#URL of page to be scraped - Syrtis
url = 'https://astrogeology.usgs.gov/search/map/Mars/Viking/valles_marineris_enhanced'
browser.visit(url)

#Pulling in data
html = browser.html

# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(html, 'html.parser')

#Image url
valles_url = soup.find('div', class_='downloads')
link = valles_url.find('a')
valles_href = link['href']

print(valles_href)

In [ ]:
#Mars Hemispheres
#Library of hemisphere images

hemisphere_image_urls = [
    {'title': 'Valles Marineris Hemisphere', 'img_url': valles_href},
    {'title': 'Syrtis Major Hemisphere', 'img_url': syrtis_href},
    {'title': 'Schiaparelli Hemisphere', 'img_url': schia_href},
    {'title': 'Cerberus Hemisphere', 'img_url': cerberus_href}
]

hemisphere_image_urls